<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></div>

In [1]:
import configparser
import os
from joblib import dump, load
import json
from tqdm import tqdm
#from helpers.helper_functions import *
#from helpers.helper_classes import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from joblib import dump, load

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)

In [3]:
# # Load data
df = pd.read_parquet('../data/training_set.parquet', engine = 'auto')
df_test = pd.read_parquet('../data/test_set.parquet', engine = 'auto')

# mini df for testing quickly 
df_mini = df[df['srch_id'] < 1000]
# df.drop('Unnamed: 0', axis=1, inplace=True)

# time to datetime
# df['time'] = pd.to_datetime(df['time'])


In [4]:
def construct_target(row):
    if row['booking_bool'] == 1:
        return 5
    elif row['click_bool'] == 1:
        return 1
    else:
        return 0

def binary_target(row):
    if row['booking_bool'] == 1:
        return 1
    else:
        return 0
    
def construct_target_df(df):
    df.loc[:, 'target'] = np.zeros(len(df))
    df.loc[df['click_bool'] == 1, 'target'] = 1
    df.loc[df['booking_bool'] == 1, 'target'] = 5
    return df['target']

def drop_cols(df, cols):
    return df.drop(cols, axis=1)

In [5]:
df_1 = df_mini.copy()
df_t = df_test.copy()

print('copied')
# Construct target and drop leaky columns

construct_target_df(df_1)
# df_1['grades'] = construct_target_df(df_1)

def drop_cols(df, cols):
    # check which cols are in dff
    cols = [col for col in cols if col in df.columns]
    return df.drop(cols, axis=1)

def construct_datetime(df):
    df_out = df.copy()
    df_out['date_time'] = pd.to_datetime(df_out['date_time'])
    df_out['month'] = df_out['date_time'].dt.month
    df_out['day'] = df_out['date_time'].dt.day
    df_out['hour'] = df_out['date_time'].dt.hour
    return df_out

def preprocess(df, train_set = True):
    leaky_cols = ['click_bool', 'booking_bool', 'gross_bookings_usd', 'position', 'date_time', 'target']

    df_out = df.copy()
    df_out = construct_datetime(df_out)
    if train_set:
        df_out['grades'] = construct_target_df(df_out)
    df_out = drop_cols(df_out, leaky_cols)
    df_out.fillna(-1, inplace=True)
    return df_out

df_1 = preprocess(df_1)
df_t = preprocess(df_t, train_set=False)
df_t
# # Construct datetime features
# df_1 = construct_datetime(df_1)
# df_t = construct_datetime(df_t)

# df_1.loc[:, 'grades'] = df_1['target']

# # Drop leaky columns



# leaky_cols = ['click_bool', 'booking_bool', 'gross_bookings_usd', 'position', 'date_time', 'target']
# df_1 = drop_cols(df_1, leaky_cols)


# ## Test: Drop all columns with NaNs
# # df_1 = df_1.dropna(axis=1)
# df_1.fillna(0, inplace=True)



# # Construct dummy cols
# dummy_cols = ['prop_country_id', 'visitor_location_country_id', 'srch_destination_id']
# df_1 = pd.get_dummies(df_1, columns=dummy_cols, drop_first=True)

# df_1

copied


,srch_id,site_id,visitor_location_country_id,visitor_hist_starrating,visitor_hist_adr_usd,prop_country_id,prop_id,prop_starrating,prop_review_score,prop_brand_bool,prop_location_score1,prop_location_score2,prop_log_historical_price,price_usd,promotion_flag,srch_destination_id,srch_length_of_stay,srch_booking_window,srch_adults_count,srch_children_count,srch_room_count,srch_saturday_night_bool,srch_query_affinity_score,orig_destination_distance,random_bool,comp1_rate,comp1_inv,comp1_rate_percent_diff,comp2_rate,comp2_inv,comp2_rate_percent_diff,comp3_rate,comp3_inv,comp3_rate_percent_diff,comp4_rate,comp4_inv,comp4_rate_percent_diff,comp5_rate,comp5_inv,comp5_rate_percent_diff,comp6_rate,comp6_inv,comp6_rate_percent_diff,comp7_rate,comp7_inv,comp7_rate_percent_diff,comp8_rate,comp8_inv,comp8_rate_percent_diff,month,day,hour
0,1,24,216,-1.0,-1.0,219,3180,3,4.5,1,2.94,0.0691,5.03,119.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
1,1,24,216,-1.0,-1.0,219,5543,3,4.5,1,2.64,0.0843,4.93,118.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
2,1,24,216,-1.0,-1.0,219,14142,2,3.5,1,2.71,0.0556,4.16,49.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,1.0,0.0,10.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
3,1,24,216,-1.0,-1.0,219,22393,3,4.5,1,2.40,0.0561,5.03,143.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
4,1,24,216,-1.0,-1.0,219,24194,3,4.5,1,2.94,0.2090,4.72,79.00,0,19222,1,10,2,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,2,2,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4959178,332787,24,216,-1.0,-1.0,117,32019,4,3.5,0,2.48,0.0551,4.53,66.07,0,19246,2,7,1,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,1.0,0.0,22.0,1.0,0.0,127.0,-1.0,0.0,27.0,1.0,0.0,22.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,21,11
4959179,332787,24,216,-1.0,-1.0,117,33959,4,3.0,1,2.20,0.3344,4.39,67.10,0,19246,2,7,1,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,0.0,0.0,-1.0,0.0,0.0,-1.0,0.0,0.0,16.0,1.0,0.0,22.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,21,11
4959180,332787,24,216,-1.0,-1.0,117,35240,4,0.0,0,1.79,-1.0000,4.64,73.91,0,19246,2,7,1,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,1.0,0.0,55.0,0.0,0.0,-1.0,0.0,0.0,16.0,0.0,0.0,3.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,21,11
4959181,332787,24,216,-1.0,-1.0,117,94437,4,0.0,0,2.94,0.0928,4.64,66.07,0,19246,2,7,1,0,1,0,-1.0,-1.0,0,-1.0,-1.0,-1.0,1.0,0.0,43.0,1.0,0.0,43.0,-1.0,0.0,12.0,-1.0,0.0,12.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,5,21,11


In [13]:
# Perform logistic regression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GroupShuffleSplit
from sklearn.metrics import roc_auc_score

def train_test_split(df, target_str, test_size=.2):
    splitter = GroupShuffleSplit(test_size=test_size, n_splits=2, random_state = 7)
    split = splitter.split(df, groups=df['srch_id'])
    train_inds, test_inds = next(split)

    df_ideal = df.iloc[test_inds].copy().sort_values(by=['srch_id', 'grades'], ascending=[True, False], inplace=False)

    X = df_1.drop(['grades'], axis=1)
    y = df_1[target_str]
    X_train, X_test, y_train, y_test, test_ideal = X.iloc[train_inds], X.iloc[test_inds], y.iloc[train_inds], y.iloc[test_inds], df_ideal, 


    return X_train, X_test, y_train, y_test, test_ideal[['srch_id', 'prop_id', 'grades']]

def construct_pred_ideal(df_in, df_ideal, y_pred):
    df = df_in.copy()
    df['pred_grades'] = y_pred
    df = df.sort_values(by=['srch_id', 'pred_grades'], ascending=[True, False], inplace=False)

    # Merge grades from ideal on srch_id and prop_id
    df = df.merge(df_ideal, on=['srch_id', 'prop_id'], how='left')

    # Return srch_id, prop_id and pred_grades
    return df[['srch_id', 'prop_id', 'pred_grades', 'grades']]

def construct_pred_submission(df_in, y_pred):
    df = df_in.copy()
    df['pred_grades'] = y_pred
    df = df.sort_values(by=['srch_id', 'pred_grades'], ascending=[True, False], inplace=False)

    # Return srch_id, prop_id and pred_grades
    return df[['srch_id', 'prop_id']]

def constructs_predictions(model, data, ideal_df = None):
    y_pred = model.predict_proba(data)
    print(y_pred)
    pred_grades = y_pred @ [0, 1, 5]

    if ideal_df is not None:
        pred_df = construct_pred_ideal(data, test_ideal, pred_grades)
    else:
        pred_df = construct_pred_submission(data, pred_grades)
    return pred_df


def calc_NDCG(df_ideal, df_pred, k = 5):
    # Group by 5
    df_ideal = df_ideal.groupby('srch_id').head(k)
    df_pred = df_pred.groupby('srch_id').head(k)

    assert df_ideal.shape[0] % k == 0
    assert df_pred.shape[0] % k == 0
    
    # Get grades matrices
    ideal_grades = df_ideal['grades'].values.reshape(int(df_ideal.shape[0] / k), k)
    pred_grades = df_pred['grades'].values.reshape(int(df_pred.shape[0] / k), k)

    discount_vec = [1/np.log2(i+2) for i in range(k)]

    # Calculate NDCG
    NDCG = (pred_grades @ discount_vec).sum() / (ideal_grades @ discount_vec).sum()

    return NDCG

X_train, X_test, y_train, y_test, test_ideal = train_test_split(df_1, 'grades')

In [14]:

# Train KNN classifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_auc_score


# Random forest
from sklearn.ensemble import RandomForestClassifier
print('Training RF')
rf = RandomForestClassifier(n_estimators=100, n_jobs=-1)
rf.fit(X_train, y_train)
print('Training RF done')
# XGBoost
print('Training XGB')
import xgboost as xgb
y_train_xgb = y_train.astype(int)
y_train_xgb[y_train == 5] = 2
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(X_train, y_train_xgb)

pred_ideal_rf = constructs_predictions(rf, X_test, ideal_df=test_ideal)
pred_xgb = constructs_predictions(xgb_model, X_test, ideal_df=test_ideal)
pred_random = construct_pred_ideal(X_test, test_ideal, np.random.rand(len(X_test)))

Training RF
Training RF done
Training XGB
[[0.82 0.03 0.15]
 [0.91 0.02 0.07]
 [0.87 0.01 0.12]
 ...
 [0.91 0.05 0.04]
 [0.98 0.   0.02]
 [0.95 0.   0.05]]
[[9.3198925e-01 1.8355350e-03 6.6175185e-02]
 [9.8796976e-01 2.9374450e-04 1.1736493e-02]
 [9.8444319e-01 3.9196028e-05 1.5517594e-02]
 ...
 [9.9400604e-01 1.2464769e-03 4.7474066e-03]
 [9.9927872e-01 1.6584058e-04 5.5547443e-04]
 [9.9859852e-01 6.4389809e-04 7.5763126e-04]]


In [10]:
# Optimize XGB with optuna
import optuna
from functools import partial

def objective(trial, X_train, y_train, X_test, test_ideal):
    y_train_xgb = y_train.astype(int)
    y_train_xgb[y_train == 5] = 2

    params = {
        "objective": "multi:softprob",
        "random_state": 42,
        "n_estimators": trial.suggest_int("n_estimators", 50, 500),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 1e-4, 1e-1, log=True),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 1),
        "reg_alpha": trial.suggest_float("reg_alpha", 1e-4, 1e-1, log=True),
        "reg_lambda": trial.suggest_float("reg_lambda", 1e-4, 1e-1, log=True),
    }

    xgb_model = xgb.XGBClassifier(**params)
    xgb_model.fit(X_train, y_train_xgb)

    pred_xgb = constructs_predictions(xgb_model, X_test, ideal_df=test_ideal)
    ndcg = calc_NDCG(test_ideal, pred_xgb)

    return ndcg

print("Training XGB")
# Assuming you have defined X_train, y_train, X_test, and test_ideal before this point.

# Wrap the objective function with the input data
objective_with_data = partial(objective, X_train=X_train, y_train=y_train, X_test=X_test, test_ideal=test_ideal)

# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction="maximize")
study.optimize(objective_with_data, n_trials=20)



[I 2023-05-10 13:48:12,402] A new study created in memory with name: no-name-368c44cd-29ec-4a52-b2d3-ee70724c2673


Training XGB


[I 2023-05-10 13:48:26,875] Trial 0 finished with value: 0.2835377374577516 and parameters: {'n_estimators': 397, 'max_depth': 8, 'learning_rate': 0.005865620161844905, 'subsample': 0.5591166538782661, 'colsample_bytree': 0.7765689031541072, 'gamma': 0.23306972001467186, 'reg_alpha': 0.03758433948837836, 'reg_lambda': 0.0006151075107843463}. Best is trial 0 with value: 0.2835377374577516.
[I 2023-05-10 13:48:32,591] Trial 1 finished with value: 0.2954637771224445 and parameters: {'n_estimators': 235, 'max_depth': 5, 'learning_rate': 0.012177280187933247, 'subsample': 0.7974999071489464, 'colsample_bytree': 0.8213446084389026, 'gamma': 0.3590176451832897, 'reg_alpha': 0.00643802195142503, 'reg_lambda': 0.002244028943139623}. Best is trial 1 with value: 0.2954637771224445.
[I 2023-05-10 13:48:40,324] Trial 2 finished with value: 0.28000904680781613 and parameters: {'n_estimators': 268, 'max_depth': 8, 'learning_rate': 0.016485889708993194, 'subsample': 0.5203336047670324, 'colsample_bytr

In [11]:
y_train_xgb = y_train.astype(int)
y_train_xgb[y_train == 5] = 2

# Get the best hyperparameters and train the final model
best_params = study.best_params
xgb_model_optimized = xgb.XGBClassifier(objective="multi:softprob", random_state=42, **best_params)
xgb_model_optimized.fit(X_train, y_train_xgb)

# Evaluate the optimized model
pred_xgb_optimized = constructs_predictions(xgb_model_optimized, X_test, ideal_df=test_ideal)
pred_xgb_submission = constructs_predictions(xgb_model_optimized, df_t)
print(f"XGB Optimized: {calc_NDCG(test_ideal, pred_xgb_optimized)}")

XGB Optimized: 0.34266561459663625


## Evaluation


In [12]:
print(f"RF: {calc_NDCG(test_ideal, pred_ideal_rf)}\n,XGB: {calc_NDCG(test_ideal, pred_xgb)},\nRandom: {calc_NDCG(test_ideal, pred_random)}")

RF: 0.29245436569120425
,XGB: 0.27329695746180704,
Random: 0.13104183655182425


In [98]:
# pred_submission.to_csv(config['PATH']['DATA_DIR'] + '/submission_RF.csv', index=False)
pred_xgb_submission.to_csv(config['PATH']['DATA_DIR'] + '/submission_XGB.csv', index=False)

In [99]:
pred_xgb_submission

,srch_id,prop_id
23,1,99484
9,1,54937
12,1,61934
5,1,28181
13,1,63894
...,...,...
4959177,332787,29018
4959178,332787,32019
4959181,332787,94437
4959182,332787,99509
